In [5]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import skfuzzy as fuzz
from sklearn.metrics import fowlkes_mallows_score
import math

# wczytanie danych

In [38]:
pd.options.display.max_rows = 6

ratings = pd.read_csv(Path('..','..','res','data','ratings.csv'))
users = pd.read_csv(Path('..','..','res','data','users.csv'))
movies = pd.read_csv(Path('..','..','res','data','movies.csv'))


# zmiana napisów na liczby
label_encoder = LabelEncoder()
users['gender'] = label_encoder.fit_transform(users['gender'])
users['occupation'] = label_encoder.fit_transform(users['occupation'])


matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')
matrix = matrix.fillna(0)

train_set, test_set = train_test_split(matrix, test_size=0.2)#, random_state=42)

users_train = users[~users['user_id'].isin(test_set.index.to_list())]
users_test = users[users['user_id'].isin(test_set.index.to_list())]

items_train = ratings[~ratings['user_id'].isin(test_set.index.to_list())]
#jeżeli jakiś przedmiot wpadnie tylko do zbioru testowego to "wypada" z dalszych analiz
#ponieważ ma przed wejściem zimnych userów sume ratingów = 0, więc zgodnie z moją metodą
#nie zostanie zarekomendowany w topN filmów
items_test = ratings[ratings['user_id'].isin(test_set.index.to_list())]

train_set.to_csv(Path('..','..','outputs','train_set.csv'), index = False)
test_set.to_csv(Path('..','..','outputs','test_set.csv'), index = False)

all_users_clustered = users_train_clusters = cluster_users(users_train.copy(), users_test.copy())
users_train_clustered = all_users_clustered[1]
users_test_clustered = all_users_clustered[0]
#items_clustered = cluster_items(items_train)
#items_clustered.to_csv(Path('..','..','outputs','items_clustered3.csv'), index=False)
items_clustered = items_train.copy()
items_clustered['cluster'] = 1


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 37, cost: 1213.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 76, cost: 1191.0
Run 2, iteration: 2/100, moves: 5, cost: 1191.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 13, cost: 1199.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 4, cost: 1254.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 319, cost: 1145.0
Run 5, iteration: 2/100, moves: 26, cost: 1145.0
Best run was number 5


In [7]:
def cluster_items(items_info):
    data_to_cluster = items_info['rating'].to_numpy()
    data_to_cluster = data_to_cluster.reshape(-1,1)
    K_FCM = 100
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data_to_cluster.T, K_FCM, 2, error=0.005, maxiter=1000)
    clusters = np.argmax(u, axis=0)
    items_info['cluster'] = clusters
    return(items_info)


def cluster_items2(items_info):
    data_to_cluster = items_info[['rating', 'user_id', 'movie_id']].to_numpy()
    print(items_info.shape, data_to_cluster.shape)
    data_to_cluster = data_to_cluster.reshape(-1,3)
    print(data_to_cluster.shape)
    K_FCM = 100
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(data_to_cluster.T, K_FCM, 2, error=0.005, maxiter=1000)
    clusters = np.argmax(u, axis=0)
    print(len(clusters))
    items_info['cluster'] = clusters
    return(items_info)

"""cluster_items_3cols = cluster_items2(items_test)"""

'cluster_items_3cols = cluster_items2(items_test)'

In [21]:
def cluster_users(users_info, test_users_info):
    data_to_cluseter = users_info[['age','gender','occupation']]
    kmode = KModes(n_clusters=5, init = "random", n_init = 5, verbose=1, random_state=42) 
    clusters = kmode.fit_predict(data_to_cluseter) 
    users_info['cluster'] = clusters
    data_to_cluster_test = test_users_info[['age','gender','occupation']]
    test_clusters = kmode.predict(data_to_cluster_test)
    test_users_info['cluster'] = test_clusters
    return(users_info, test_users_info)

In [39]:
def IP(train_matrix, items_clustered):

    train_numpy = train_matrix.to_numpy()
    
    #szukanie IPmax
    IP = np.sum(train_numpy, axis = 0) #sumowanie po kolumnach
    IP = IP.reshape(1,-1)
    IPmax = np.amax(IP)
    IPmax_coordinates = np.argmax(IP) + 1 #bo filmy mają indeksy od 1
    item_rating = pd.DataFrame({'rating':IP[0], 'movie_id': train_matrix.columns.to_list()})
    
    #znalezienie filmu o najwyższym IP
    IPmax_movie_id = item_rating[item_rating['rating'] == IPmax]['movie_id'].values[0]
    
    #znalezienie clustra, gdzie znajduje się ten film i pozostałych należących tu filmów
    IPmax_cluster = items_clustered[items_clustered['movie_id'] == IPmax_movie_id].iloc[0,4]
    max_clust_movies = items_clustered[items_clustered['cluster'] == IPmax_cluster]['movie_id'].to_list()
    max_clust_movies = list(set(max_clust_movies))

    #obliczenie IP^
    IP_dash = IP[:,np.array(max_clust_movies) - 1] / IPmax
    
    return(IP_dash, max_clust_movies)

IP_dash_movies = IP(train_set, items_clustered)
IP_dashhhh = IP_dash_movies[0]
IP_moviesss = IP_dash_movies[1]

1


In [10]:
#def UP_HP(train_test_set, cluster, clust_data, IP_dash, alpha, selected_movies, ratings_matrix):
def UP_HP(train_matrix, test_matrix, test_users_cl, train_users_cl, cluster, IP_dash, alpha, selected_movies, ratings_matrix):    
    
    #utworzenie macierzy, gdzie ratingi dane przez testowych są zerami
    test_set_zero = test_matrix.copy()
    test_set_zero[:] = 0 
    train_and_test = pd.concat([train_matrix, test_set_zero], ignore_index=False)
    train_and_test = train_and_test.sort_index()
    
    concated_users_clustered = pd.concat([train_users_cl,test_users_cl], ignore_index=True).sort_values('user_id')

    #znalezienie użytkowników w danym clustrze
    users_in_cluster = concated_users_clustered[concated_users_clustered['cluster'] == cluster]['user_id'].to_list()
    users_in_cluster = list(set(users_in_cluster))
    
    #macierz ratingów tylko dla tych userów i filmów z clustra z IPmax
    ratings_matrix_cluster = train_and_test.loc[train_and_test.index.isin(users_in_cluster),selected_movies].sort_index()
    ratings_matrix_cluster_np = ratings_matrix_cluster.to_numpy()

    #obliczanie user popularity
    UP = np.sum(ratings_matrix_cluster_np, axis=1) ##sumowanie po wierszach
    UP = UP.reshape(-1,1)
    UPmax = np.amax(UP)
    UPmax_coordinates = np.argmax(UP)
    UP_dash = UP / UPmax
    
    # obliczenie UIP i hybrydy
    UIP = UP_dash / IP_dash
    #wstawienie zer tam gdzie w rzeczywistości uzytkownik treningowy nie ocenił danego filmu
    UIP = np.where(ratings_matrix_cluster == 0, 0, UIP)
    HP = alpha * IP_dash + (1-alpha) * np.sum(UIP,axis = 0).reshape(1,-1)
    return(HP)
    #return np.sum(pd.DataFrame(UIP, index=ratings_matrix_cluster.index, columns=ratings_matrix_cluster.columns).loc[243,9])

"""test_set_zero = test_set.copy()
test_set_zero[:] = 0
concated_users_clustered = pd.concat([us_tr_cl,us_te_cl], ignore_index=True).sort_values('user_id')
train_and_test = pd.concat([train_set, test_set_zero], ignore_index=False)
train_and_test = train_and_test.sort_index()"""

"""UP_HP(train_set, test_set, us_te_cl, us_tr_cl, 1, IP_dashhhh, 0.5, IP_moviesss, ratings_matrix)"""

'UP_HP(train_set, test_set, us_te_cl, us_tr_cl, 1, IP_dashhhh, 0.5, IP_moviesss, ratings_matrix)'

In [11]:
def HP_for_clust(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    
    #zapewnienie prawdiłowej kolejnosci wierszy i kolumn
    train_matrix = train_matrix.sort_index()
    test_matrix = test_matrix.sort_index()
    
    #podział na clustry
    #users_clustered = cluster_users(users_train.copy())
    #items_clustered = cluster_items(items_train.copy())
    
    """users_clustered = pd.read_csv(Path('..','..','outputs','users_train_clustered.csv'))
    #items_clustered = pd.read_csv(Path('..','..','outputs','items_train_clustered.csv'))
    users_clustered['cluster'] = 1
    #przypisanie nowych użytkowników do clustrów
    kmode = KModes(n_clusters=5, init = "random", n_init = 5, random_state = 42)#verbose=1, random_state=42) 
    train_clusters = kmode.fit_predict(users_train)
    test_users_clusters = kmode.predict(users_test)
    test_users_clustered = users_test.copy() ##OCEŃ CZY KOPIOWANIE JEST KONIECZNE
    test_users_clustered['cluster'] = test_users_clusters"""
    
    ### klastrowanie ###
    test_users_clustered = test_users_cl
    test_users_clusters = test_users_clustered['cluster'].values
    items_clustered = items_cl
    
    ## IP ##
    IP_results = IP(train_matrix, items_clustered)
    IP_dash = IP_results[0]
    movies_from_max_cluster = IP_results[1]

    clust_HP = {}
    for cl in list(set(test_users_clusters)):
        HP = UP_HP(train_matrix, test_matrix, test_users_clustered, train_users_cl, cl, IP_dash, alpha, movies_from_max_cluster, ratings_matrix)[0].tolist()  
        clust_HP[cl] = (movies_from_max_cluster, HP)
    return(clust_HP, test_users_clustered) #słownik [cluster[]: [filmy z clustra gdzie IP_max, HP],
                                            #testowi uzytkownicy z wygenerowanymi klastrami

"""z = HP_for_clust(test_matrix, train_matrix, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)
len(z[0][1][0]) #długość movies_from_max_cluster
z[0][1][1][-3] #3 od konca wartość z HP"""

'z = HP_for_clust(test_matrix, train_matrix, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)\nlen(z[0][1][0]) #długość movies_from_max_cluster\nz[0][1][1][-3] #3 od konca wartość z HP'

In [12]:
def create_matrix_for_generated(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    HP_and_clust = HP_for_clust(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl)
    movies_and_HP = HP_and_clust[0] #słownik [cluster]:[filmy do polecenia, HP]
    
    #dataframe testowych uzytkownikow z przypisanymi clustrami
    test_users_with_clusters = HP_and_clust[1]
    
    #dataframe pokazujący cluster i ilu uzytkowników do niego nalezy
    num_users_in_clust = test_users_with_clusters.groupby('cluster').agg(users_number = ('cluster','size')).reset_index()
    num_users_in_clust_dict = num_users_in_clust.set_index('cluster')['users_number'].to_dict() #liczba użytkowników w każdym clustrze

    #słownik cluster: id uzytkownikow którzy do niego nalezą
    users_per_cluster = test_users_with_clusters.groupby('cluster')['user_id'].apply(list).to_dict() #uzytkownicy (id) w kazdym clustrze

    test_users_dictionary = {'user_id':[],
                         'movie_id':[],
                         'rating':[]}
    
    for cl in users_per_cluster.keys():
        num_of_users = num_users_in_clust_dict[cl]
        num_of_items = len(movies_and_HP[cl][0])

        multiplied_users = users_per_cluster[cl] * num_of_items
        multiplied_users.sort()
        test_users_dictionary['user_id'].extend(multiplied_users)
        #aktualizuję uzytkowników w clustrze powielając ich tyle razy ile jest przedmiotów, 
        #bo każdy musi dany przedmiot mieć oceniony
        users_per_cluster[cl] = multiplied_users

        #powielam przedmioty i oceny tyle razy ilu jest użytkowników
        movies_and_HP[cl] = (movies_and_HP[cl][0] * num_of_users, movies_and_HP[cl][1] * num_of_users)
        
        test_users_dictionary['movie_id'].extend(movies_and_HP[cl][0])
        test_users_dictionary['rating'].extend(movies_and_HP[cl][1])

        test_ratings_HP = pd.DataFrame(test_users_dictionary)
        test_ratings_HP.to_csv(Path('..','..','outputs','test_ratings.csv'), index=False)
        test_ratings_matrix = test_ratings_HP.pivot(index='user_id', columns='movie_id', values='rating')
        test_ratings_matrix = test_ratings_matrix.fillna(0)

    return test_ratings_matrix

"""create_matrix_for_generated(test_set, train_set, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)
"""

'create_matrix_for_generated(test_set, train_set, 0.5, ratings_matrix, us_te_cl, us_tr_cl, items_clusters)\n'

In [13]:
def show_Hu(row):
    #wybieranie id filmów, dla których w zbiorze testowym użytkownik wystawił jakieś oceny
    nonzero = row[row > 0].index
    nonzero_indices = nonzero.values
    row['Hu'] = nonzero_indices.tolist()
    return row

In [14]:
def find_topN_items(users_ratings,N):
    topN = users_ratings.nlargest(N) #N najlepiej ocenianych filmów (rating i indeks) 
                                        #zwraca się od najwiekszej do najmn wartosci
    topN_indices = topN.index #indeksy tych najlepiej ocenianych
    #wyrzucam z topN filmy, dla których wystawiono zero
    top_nonzero_indices = users_ratings[topN_indices][users_ratings[topN_indices] > 0].index
    users_ratings["topN_movies"] = top_nonzero_indices.values
    return users_ratings

In [15]:
def NDCG(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    #sprawdzenie który n-ty element z TopN został irl oceniony przez użytkownika
    available_n = [index + 1  for index, element in enumerate(generated) if element in from_test_set]
    DCG = sum([1 / math.log(1+n,2) for n in available_n])
    IDCG = sum([math.log(1+n,2) for n in range(1,N+1)])
    NDCG = DCG / IDCG
    row['NDCG'] = NDCG
    return row

In [16]:
def Precision_Recall(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    intersection = sum([1 for element in generated if element in from_test_set])
    
    Precision = intersection * 100 / N
    row['Precision'] = Precision
    
    row['Recall'] = intersection * 100 / len(from_test_set)
    return row

In [17]:
def nazwepozniej(test_matrix, train_matrix, alpha, N, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    
    generated = create_matrix_for_generated(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl)
    test_with_Hu = test_matrix.apply(lambda row: show_Hu(row), axis = 1)
    generated_topN = generated.apply(lambda row: find_topN_items(row, N), axis = 1)
    Hu = test_with_Hu.iloc[:,[-1]].sort_index()

    merged = pd.merge(generated_topN, Hu, on='user_id', how = 'left')
    merged = merged.apply(lambda row:  NDCG(row,N),axis = 1)
    merged = merged.apply(lambda row:  Precision_Recall(row,N),axis = 1)
    
    return(merged)
"""output_final = nazwepozniej(test_set, train_set, 0.5, 50, ratings_matrix, us_te_cl, us_tr_cl, items_cl)"""

'output_final = nazwepozniej(test_set, train_set, 0.5, 50, ratings_matrix, us_te_cl, us_tr_cl, items_cl)'

In [31]:
def main(ratings_data, items_data, users_data, alpha, N):
    matrix = ratings_data.pivot(index='user_id', columns='movie_id', values='rating')
    matrix = matrix.fillna(0)
    
    train_set, test_set = train_test_split(matrix, test_size=0.2)#, random_state=42)

    users_train = users_data[~users_data['user_id'].isin(test_set.index.to_list())]
    users_test = users_data[users_data['user_id'].isin(test_set.index.to_list())]

    items_train = ratings_data[~ratings_data['user_id'].isin(test_set.index.to_list())]
    #jeżeli jakiś przedmiot wpadnie tylko do zbioru testowego to "wypada" z dalszych analiz
    #ponieważ ma przed wejściem zimnych userów sume ratingów = 0, więc zgodnie z moją metodą
    #nie zostanie zarekomendowany w topN filmów
    items_test = ratings_data[ratings_data['user_id'].isin(test_set.index.to_list())]

    train_set.to_csv(Path('..','..','outputs','train_set.csv'), index = False)
    test_set.to_csv(Path('..','..','outputs','test_set.csv'), index = False)

    all_users_clustered = users_train_clusters = cluster_users(users_train.copy(), users_test.copy())
    users_train_clustered = all_users_clustered[1]
    users_test_clustered = all_users_clustered[0]
    items_clustered = pd.read_csv(Path('..','..','outputs','items_clustered3.csv'))#cluster_items(items_train)
    items_clustered.to_csv(Path('..','..','outputs','items_clustered3.csv'), index=False)
    final_matrix = nazwepozniej(test_set, train_set, alpha, N, matrix, users_test_clustered, users_train_clustered, items_clustered)
    return(final_matrix)

In [32]:
main(ratings, movies, users, 0.5, 20)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 171, cost: 1202.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 12, cost: 1196.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 3, cost: 1242.0
Run 3, iteration: 2/100, moves: 1, cost: 1242.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 1201.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 377, cost: 1212.0
Run 5, iteration: 2/100, moves: 5, cost: 1212.0
Best run was number 2


C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: divide by zero encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: invalid value encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: divide by zero encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: invalid value encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: divide by zero encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: invalid value encountered in divide
  UIP = UP_dash / IP_dash
C:\Users\wypyc\AppData\Local\Temp\ipykernel_17572\1591590274.py:28: RuntimeWarning: divide by zero encountered in divide


TypeError: argument of type 'float' is not iterable

### do uruchomienia, jeśli chce się na sztywno clustry przypisać

In [19]:
us_te_cl = users_test.copy()
us_te_cl['cluster'] = 1

us_tr_cl = users_train.copy()
us_tr_cl['cluster'] = 1

it_cl = items_train.copy()
it_cl['cluster'] = 1

"""it_cl = article_items_cl
    it_cl"""

NameError: name 'users_test' is not defined